# Processing the Aircraft Metadata
1. Adding column names
2. Adjusting to timestamps exactly as the video files.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/harvard.dce.nasa.cloud2cloud/er2_aircraft_metadata
!ls -ltr

Mounted at /content/drive
/content/drive/MyDrive/harvard.dce.nasa.cloud2cloud/er2_aircraft_metadata
total 51051
-rw------- 1 root root 5164161 Sep 19 14:32 IWG1.18Apr2017-2321.txt
-rw------- 1 root root 4992950 Sep 19 14:34 IWG1.23Apr2017-0142.txt
-rw------- 1 root root 7276927 Sep 19 14:36 IWG1.09May2017-0412.txt
-rw------- 1 root root 6353641 Sep 19 14:36 IWG1.12May2017-2104.txt
-rw------- 1 root root 6454052 Sep 19 14:37 IWG1.14May2017-1922.txt
-rw------- 1 root root     367 Sep 21 14:58 IWG1.23Apr2017-0142_processed_170423_203040_170423_234813.csv
-rw------- 1 root root 2500765 Sep 21 15:09 IWG1.23Apr2017-0142_processed_170422_203040_170422_234813.csv
-rw------- 1 root root 3940991 Sep 21 15:10 IWG1.18Apr2017-2321_processed_170418_175706_170418_230811.csv
-rw------- 1 root root 4077996 Sep 21 15:10 IWG1.14May2017-1922_processed_170514_115518_170514_171917.csv
-rw------- 1 root root 3599390 Sep 21 15:10 IWG1.12May2017-2104_processed_170512_160457_170512_204918.csv
-rw------- 1 root 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import os
import io
import cv2
import time
import shutil
import datetime

In [3]:
def process_aircraft_metadata(file_path, start, end):
    """
    Read the ER2 Aircraft Metadata file.
    Align it with the start and end times.
    Add column names and make sure the records are per second.
    Write processed output data to a csv with filename f"{file_path}_processed_{start}_{end}.csv"
    Return the file path of the output file.
    Args:
        file_path (str): file path of metadata file
        start (str): start time in HHMMSS format
        end (str): end time in HHMMSS format
    Returns:
        (str): file path of output file
    """
    columns = [
      "Short_Name", "DateTime_UTC", "Lat", "Lon", "GPS_MSL_Alt", "WGS_84_Alt",
      "Press_Alt", "Radar_Alt", "Grnd_Spd", "True_Airspeed", "Indicated_Airspeed",
      "Mach_Number", "Vert_Velocity", "True_Hdg", "Track", "Drift", "Pitch",
      "Roll", "Side_slip", "Angle_of_Attack", "Ambient_Temp", "Dew_Point",
      "Total_Temp", "Static_Press", "Dynamic_Press", "Cabin_Pressure", "Wind_Speed",
      "Wind_Dir", "Vert_Wind_Spd", "Solar_Zenith", "Sun_Elev_AC", "Sun_Az_Grd",
      "Sun_Az_AC"
    ]
    # read the file
    df = pd.read_csv(file_path, names=columns, header=None, skipinitialspace=True, na_values='')
    print("raw file:", df.shape)
    df = df.where(pd.notna(df), None)
    df['DateTime_UTC'] = pd.to_datetime(df['DateTime_UTC'])
    # add column called timestamp with format YYMMDD_HHMMSS, formed using DateTime_UTC column
    df['timestamp'] = df['DateTime_UTC'].dt.strftime('%y%m%d_%H%M%S')
    print("timestamp added: ", df.shape)
    # if for a given value of timestamp there are multiple values of DateTime_UTC keep only the first one..
    df = df.drop_duplicates(subset=['timestamp'], keep='first')
    print("drop duplicates: ", df.shape)
    # filter df to include only records having timestamps '%y%m%d_%H%M%S' starting from start to end
    start_dt = datetime.datetime.strptime(start, '%y%m%d_%H%M%S')
    end_dt = datetime.datetime.strptime(end, '%y%m%d_%H%M%S')
    print("start and end date: ", start_dt, end_dt)
    df = df[(df['DateTime_UTC'] >= start_dt) & (df['DateTime_UTC'] <= end_dt)]
    print("filtered for start and end: ", df.shape)
    # extract file_path without its extension
    file_path_wo_ext = os.path.splitext(file_path)[0]
    # write file to a csv with filename f"{file_path}_processed_{start}_{end}.csv"
    output_file_path = f"{file_path_wo_ext}_processed_{start}_{end}.csv"
    df.to_csv(output_file_path, index=False)
    print("final shape: ", df.shape)
    return output_file_path


In [4]:
print(process_aircraft_metadata("IWG1.23Apr2017-0142.txt", "170422_203040", "170422_234813"))

raw file: (25916, 33)
timestamp added:  (25916, 34)
drop duplicates:  (25912, 34)
start and end date:  2017-04-22 20:30:40 2017-04-22 23:48:13
filtered for start and end:  (11853, 34)
final shape:  (11853, 34)
IWG1.23Apr2017-0142_processed_170422_203040_170422_234813.csv


In [ ]:
# read the csv and show the head
df_pr = pd.read_csv("IWG1.12May2017-2104_processed_2017-05-12 16:04:57_2017-05-12 20:49:18.csv")
df_pr.head()

,Short_Name,DateTime_UTC,Lat,Lon,GPS_MSL_Alt,WGS_84_Alt,Press_Alt,Radar_Alt,Grnd_Spd,True_Airspeed,...,Dynamic_Press,Cabin_Pressure,Wind_Speed,Wind_Dir,Vert_Wind_Spd,Solar_Zenith,Sun_Elev_AC,Sun_Az_Grd,Sun_Az_AC,timestamp
0,IWG1,2017-05-12 16:04:57.004,28.748027,-91.362701,19809.4,19809.5,64315.0,NaN,206.9,208.8,...,24.2,311.8,3.7,179.5,NaN,28.9,60.9,104.7,10.0,170512_160457
1,IWG1,2017-05-12 16:04:58.004,28.747884,-91.360687,19809.3,19809.4,NaN,NaN,206.9,208.7,...,NaN,311.9,3.7,179.3,NaN,28.9,60.9,104.7,9.9,170512_160458
2,IWG1,2017-05-12 16:04:59.004,28.747740,-91.358673,19809.3,19809.3,64315.0,NaN,206.9,208.7,...,24.2,311.9,3.8,178.1,NaN,28.9,60.9,104.7,9.8,170512_160459
3,IWG1,2017-05-12 16:05:00.004,28.747596,-91.356315,19809.3,19809.3,64317.5,NaN,207.0,208.6,...,24.2,311.9,3.9,177.9,NaN,28.9,60.9,104.7,9.9,170512_160500
4,IWG1,2017-05-12 16:05:01.004,28.747452,-91.354301,19809.4,19809.3,64317.5,NaN,207.0,208.7,...,24.2,311.5,3.9,178.3,NaN,28.9,60.9,104.7,9.9,170512_160501


In [ ]:
df_pr.tail()

,Short_Name,DateTime_UTC,Lat,Lon,GPS_MSL_Alt,WGS_84_Alt,Press_Alt,Radar_Alt,Grnd_Spd,True_Airspeed,...,Dynamic_Press,Cabin_Pressure,Wind_Speed,Wind_Dir,Vert_Wind_Spd,Solar_Zenith,Sun_Elev_AC,Sun_Az_Grd,Sun_Az_AC,timestamp
17054,IWG1,2017-05-12 20:49:13.004,32.586725,-83.545411,439.0,441.4,1495.0,NaN,48.8,45.7,...,11.8,949.1,NaN,NaN,NaN,46.4,46.9,-95.4,-55.9,170512_204913
17055,IWG1,2017-05-12 20:49:14.004,32.587078,-83.545786,436.5,439.0,1487.5,NaN,48.8,45.7,...,12.0,949.5,NaN,NaN,NaN,46.4,49.2,-95.4,-56.8,170512_204914
17056,IWG1,2017-05-12 20:49:15.004,32.587429,-83.546158,433.2,436.5,1470.0,NaN,49.1,45.5,...,11.7,950.1,NaN,NaN,NaN,46.4,49.2,-95.4,-58.2,170512_204915
17057,IWG1,2017-05-12 20:49:16.004,32.587782,-83.546524,428.0,433.2,1452.5,NaN,49.6,47.5,...,12.7,950.1,NaN,NaN,NaN,46.4,49.2,-95.4,-58.9,170512_204916
17058,IWG1,2017-05-12 20:49:17.005,32.588138,-83.546886,421.9,428.0,1427.5,NaN,50.1,48.6,...,13.5,950.8,NaN,NaN,NaN,46.4,47.7,-95.4,-58.9,170512_204917


In [ ]:
# dictionary with all IWG1 filenames
metadata_dict = {
    "IWG1.18Apr2017-2321.txt": ["170418_175706", "170418_230811"],
    "IWG1.23Apr2017-0142.txt": ["170422_203040", "170422_234813"],
    "IWG1.09May2017-0412.txt": ["170508_202828", "170509_014103"],
    "IWG1.12May2017-2104.txt": ["170512_160457", "170512_204918"],
    "IWG1.14May2017-1922.txt": ["170514_115518", "170514_171917"]
}
for filename in os.listdir("er2_aircraft_metadata"):
  if filename.endswith(".txt"):
    file_path = os.path.join("er2_aircraft_metadata", filename)
    try:
      print(process_aircraft_metadata(file_path, metadata_dict[filename][0], metadata_dict[filename][1]))
      print(f"Processed: {filename}")
    except Exception as e:
      print(f"Error processing {filename}: {e}")

(26847, 33)
(26847, 34)
(26846, 34)
2017-04-18 17:57:06 2017-04-18 23:08:11
(18665, 34)
(18665, 34)
er2_aircraft_metadata/IWG1.18Apr2017-2321_processed_170418_175706_170418_230811.csv
Processed: IWG1.18Apr2017-2321.txt
(33436, 33)
(33436, 34)
(33434, 34)
2017-05-14 11:55:18 2017-05-14 17:19:17
(19439, 34)
(19439, 34)
er2_aircraft_metadata/IWG1.14May2017-1922_processed_170514_115518_170514_171917.csv
Processed: IWG1.14May2017-1922.txt
(33047, 33)
(33047, 34)
(33041, 34)
2017-05-12 16:04:57 2017-05-12 20:49:18
(17059, 34)
(17059, 34)
er2_aircraft_metadata/IWG1.12May2017-2104_processed_170512_160457_170512_204918.csv
Processed: IWG1.12May2017-2104.txt
(37242, 33)
(37242, 34)
(37240, 34)
2017-05-08 20:28:28 2017-05-09 01:41:03
(18755, 34)
(18755, 34)
er2_aircraft_metadata/IWG1.09May2017-0412_processed_170508_202828_170509_014103.csv
Processed: IWG1.09May2017-0412.txt
(25916, 33)
(25916, 34)
(25912, 34)
2017-04-22 20:30:40 2017-04-22 23:48:13
(11853, 34)
(11853, 34)
er2_aircraft_metadata/IW

In [ ]:
df_pr = pd.read_csv("er2_aircraft_metadata/IWG1.09May2017-0412_processed_170508_202828_170509_014103.csv")
df_pr.head()

,Short_Name,DateTime_UTC,Lat,Lon,GPS_MSL_Alt,WGS_84_Alt,Press_Alt,Radar_Alt,Grnd_Spd,True_Airspeed,...,Dynamic_Press,Cabin_Pressure,Wind_Speed,Wind_Dir,Vert_Wind_Spd,Solar_Zenith,Sun_Elev_AC,Sun_Az_Grd,Sun_Az_AC,timestamp
0,IWG1,2017-05-08 20:28:28.004,36.340627,-93.153874,19324.4,19324.3,62602.5,NaN,207.6,203.6,...,24.5,317.7,4.6,85.4,NaN,35.9,53.3,-112.7,-47.9,170508_202828
1,IWG1,2017-05-08 20:28:29.004,36.341454,-93.155926,19324.4,19324.4,62602.5,NaN,207.6,203.6,...,24.5,317.3,4.6,85.3,NaN,35.9,53.3,-112.7,-47.9,170508_202829
2,IWG1,2017-05-08 20:28:30.004,36.342278,-93.157977,19324.6,19324.4,62600.0,NaN,207.6,203.6,...,24.5,318.0,4.5,83.5,NaN,35.9,53.3,-112.7,-48.0,170508_202830
3,IWG1,2017-05-08 20:28:31.004,36.343102,-93.160028,19324.7,19324.6,62602.5,NaN,207.6,203.7,...,24.5,317.2,4.6,82.6,NaN,36.0,53.3,-112.7,-48.0,170508_202831
4,IWG1,2017-05-08 20:28:32.004,36.343926,-93.162079,19324.9,19324.7,62602.5,NaN,207.6,203.6,...,24.5,317.5,4.6,82.8,NaN,36.0,53.3,-112.7,-48.0,170508_202832


In [ ]:
df_pr.tail()

,Short_Name,DateTime_UTC,Lat,Lon,GPS_MSL_Alt,WGS_84_Alt,Press_Alt,Radar_Alt,Grnd_Spd,True_Airspeed,...,Dynamic_Press,Cabin_Pressure,Wind_Speed,Wind_Dir,Vert_Wind_Spd,Solar_Zenith,Sun_Elev_AC,Sun_Az_Grd,Sun_Az_AC,timestamp
18750,IWG1,2017-05-09 01:40:58.004,38.067420,-97.305743,20854.7,20854.8,67657.5,NaN,209.5,204.6,...,19.0,303.6,4.2,282.1,NaN,93.1,-2.2,-65.0,177.6,170509_014058
18751,IWG1,2017-05-09 01:40:59.004,38.066557,-97.303737,20854.6,20854.7,67657.5,NaN,209.5,204.6,...,19.0,302.1,4.2,282.0,NaN,93.2,-2.2,-65.0,177.6,170509_014059
18752,IWG1,2017-05-09 01:41:00.004,38.065694,-97.301732,20854.5,20854.6,67652.5,NaN,209.5,204.4,...,19.0,301.7,4.1,284.5,NaN,93.2,-2.3,-65.0,177.7,170509_014100
18753,IWG1,2017-05-09 01:41:01.005,38.064835,-97.299724,20854.3,20854.5,67657.5,NaN,209.5,204.6,...,19.0,301.7,4.0,285.7,NaN,93.2,-2.3,-65.0,177.7,170509_014101
18754,IWG1,2017-05-09 01:41:02.004,38.063972,-97.297375,20854.1,20854.3,67657.5,NaN,209.6,204.6,...,19.0,302.2,4.0,285.8,NaN,93.2,-2.3,-65.0,177.7,170509_014102
